# Frequency Selection Tool for Undersampling Multisine Signals

In [ ]:
%matplotlib widget

import multisine as ms
from multisine import np
from multisine import plt
from multisine import mpl
from multisine import cm
from multisine import nx

## if you have installed latex and want to use it for plots, uncomment the following 4 lines
# plt.rcParams["text.usetex"] = True
# mpl.rcParams.update({"text.usetex": True, "savefig.format": "pdf"})
# mpl.rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
# mpl.rc("text.latex", preamble=r"\usepackage{underscore}")

## safe figures e.g. with:
# plot_name = "adjacency_matrix"
# plt.savefig(r"figures/" + plot_name + ".pdf", dpi=600)

In [ ]:
f_s = 24
f_lpf = 70
harmonics_measurable = 3
harmonics_expected = 3

In [ ]:
f_max = np.int64(np.floor(f_lpf/(harmonics_measurable)))
f_alias = f_s / 2
f_max_example = f_max + 2

## Introduction

The key problem to be solved is, that frequencies can overlap if the lowpass filter is not matching the sampling rate.  
We assume, that we are looking for a multisine signal.

In [ ]:
fig, axes = ms.draw_signal_path()

If a measured frequency is above the Nyquist frequency, it is aliased into the first Nyquist zone

In [ ]:
fig, axes = ms.plot_example_undersampling()

If a nonlinear system is measured, such as a battery, the system might have further harmonics. All harmonics above the ideal LFP frequency are filtered out.

In [ ]:
fig, axes = ms.plot_example_bins()

Let's use an other representation.  
If the system to be analysed is linear and the sampling rate **f_s** and lowpass filter **f_lpf** are high enough, the Input Frequency maps directly to the Output Frequency:

In [ ]:
f_raw = ms.get_raw_frequencies(f_max_example)
fig, ax = plt.subplots(1,1,figsize=(8*cm, 6*cm))
ax.plot(f_raw)
ax.set_xlim(0, f_max_example)
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.set_yticks(np.concat([ax.get_yticks(), [f_lpf, f_alias]]), np.concat([ax.get_yticklabels(), ['$f_\mathrm{lpf}$','$f_\mathrm{N}$']]))
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.grid(True)
ax.set_xlabel('Input Frequency\n in Hz')
ax.set_ylabel('Output Frequency\n in Hz')
ax.legend(['1. Harmonic'], loc='upper left',handlelength=0.2)
fig.tight_layout()

If the system is nonlinear, harmonics occures. If we assume, we know how many harmonics our systems creates (**harmonics_expected**), we can decide how many of these we want to measure (**harmonics_measurable**).

In [ ]:
f_harm = ms.get_harmonics(f_raw,harmonics_expected)
fig, ax = plt.subplots(1,1,figsize=(8*cm, 6*cm))
ax.plot(f_harm)
ax.set_xlim(0, f_max_example)
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.set_yticks(np.concat([ax.get_yticks(), [f_lpf, f_alias]]), np.concat([ax.get_yticklabels(), ['$f_\mathrm{lpf}$','$f_\mathrm{N}$']]))
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.grid(True)
ax.set_xlabel('Input Frequency\n in Hz')
ax.set_ylabel('Output Frequency\n in Hz')
ax.legend(['_Hidden', '2. Harmonic', '3. Harmonic'], loc='upper left',handlelength=0.2)
fig.tight_layout()

Most system have a lowpass filter:

In [ ]:
f_lowp = ms.low_pass_filter(f_harm,f_lpf)
fig, ax = plt.subplots(1,1,figsize=(8*cm, 6*cm))
ax.plot(f_lowp)
ax.set_xlim(0, f_max_example)
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.set_yticks(np.concat([ax.get_yticks(), [f_lpf, f_alias]]), np.concat([ax.get_yticklabels(), ['$f_\mathrm{lpf}$','$f_\mathrm{N}$']]))
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.grid(True)
ax.set_xlabel('Input Frequency\n in Hz')
ax.set_ylabel('Output Frequency\n in Hz')
fig.tight_layout()

And all digital systems have a sample rate, that maps frequencies higher than f_s/2 to other frequencies:

In [ ]:
f_digital = ms.get_alias_f(f_lowp,f_s)
fig, ax = plt.subplots(1,1,figsize=(8*cm, 6*cm))
ax.plot(f_digital)
ax.set_xlim(0, f_max_example)
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.set_yticks(np.concat([ax.get_yticks(), [f_lpf, f_alias]]), np.concat([ax.get_yticklabels(), ['$f_\mathrm{lpf}$','$f_\mathrm{N}$']]))
ax.set_ylim(0, f_max_example*max(harmonics_expected, harmonics_measurable))
ax.grid(True)
ax.set_xlabel('Input Frequency\n in Hz')
ax.set_ylabel('Output Frequency\n in Hz')
fig.tight_layout()

Another way of visulasation is below.

In [ ]:
harmonics = ms.get_raw_frequencies(f_s*harmonics_expected)
harmonics = ms.get_harmonics(harmonics,harmonics_expected)
harmonics = ms.low_pass_filter(harmonics,f_lpf)
harmonics = ms.get_alias_f(harmonics,f_s)

fig, ax = plt.subplots(figsize=(16*cm, 8*cm))
CS = ax.contourf(np.transpose(harmonics),cmap = plt.get_cmap('turbo'))
cbar = fig.colorbar(CS, shrink=1, extend='both', ticks=[0,f_alias])
cbar.ax.set_yticklabels(['$0$','$f_N$'])
cbar.set_label('Output Frequency in Hz')
CS.set_edgecolor('face')
ax.set_xlabel('Input Frequency in Hz')
ax.set_ylabel('Harmonic Order')
ax.grid()

ax.set_xticks([0, f_alias,  f_alias*2,f_max-1,f_alias*3,f_alias*4,f_alias*5,f_alias*6,f_lpf-1], ['$0$', '$f_N$', '$f_s$', '\n$f_\mathrm{max}$','','$2f_s$','','$3f_s$','\n$f_\mathrm{LPF}$'], fontsize=10);
ax.set_yticks([0, 1, 2],['$1$', '$2$', '$3$']);

fig.tight_layout()

## Frequency search

If we know have all frequencies that are possible and their harmonics, we can first see, if they overlap. In an adjacence matrix we store with a 1 if two frequencies can be excited at the same time or not.

In [ ]:
frequencies = ms.get_frequencies(f_s,f_lpf,harmonics_measurable,harmonics_expected)

In [ ]:
adjacency_mat = ms.get_adjacency(f_s,f_lpf,harmonics_measurable,frequencies)

In [ ]:
f_nyquist = (f_s)/2
f_nyquist_int = int(np.floor(f_nyquist))

fig, ax = plt.subplots(1,1,figsize=(8*cm, 8*cm))

adjacency_mat_plot = adjacency_mat.copy()

if len(adjacency_mat_plot) > f_max:
    adjacency_mat_plot[f_max:,:] = 0 
    adjacency_mat_plot[:,f_max:] = 0

CS = ax.imshow(adjacency_mat_plot,cmap='Greys')
ax.set_xticks([0, f_nyquist, f_lpf, f_max, f_s], ['$0$', '$f_{\mathrm{N}}$', '$f_{\mathrm{LPF}}$','$f_{\mathrm{max}}$\n', '$f_{\mathrm{s}}$'])
ax.set_yticks([0, f_nyquist, f_lpf, f_max, f_s], ['$0$', '$f_{\mathrm{N}}$', '$f_{\mathrm{LPF}}$','$f_{\mathrm{max}}$\n', '$f_{\mathrm{s}}$'])
ax.grid()

plt.xlabel('$\mathrm{Frequency}\  \mathrm{in\ Hz}$')
plt.ylabel('$\mathrm{Frequency}\  \mathrm{in\ Hz}$')
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')

ax.plot([f_nyquist, f_s], [f_nyquist, f_nyquist], 'C0-', lw=1)
ax.plot([f_s, f_s], [f_nyquist, f_s], 'C0-', lw=1)
ax.plot([f_s, f_nyquist], [f_s, f_nyquist], 'C0-', lw=1)

ax.plot([f_nyquist, f_s], [f_nyquist, f_nyquist], 'C1-', lw=1)
ax.plot([f_s, f_s], [f_nyquist, 0], 'C1-', lw=1)
ax.plot([f_s, f_nyquist], [0, f_nyquist], 'C1-', lw=1)

ax.plot([f_nyquist, f_s], [f_nyquist, 0], 'C2-', lw=1)
ax.plot([f_s, f_nyquist], [0, 0], 'C2-', lw=1)
ax.plot([f_nyquist, f_nyquist], [0, f_nyquist], 'C2-', lw=1)

ax.plot([0, f_nyquist], [0, f_nyquist], 'C3-', lw=1)
ax.plot([f_nyquist, f_nyquist], [f_nyquist, 0], 'C3-', lw=1)
ax.plot([f_nyquist, 0], [0, 0], 'C3-', lw=1)

legend_elements = [
    plt.Line2D([0], [0], marker='s', color='black',
               label='Possible\n frequency\n combination', alpha=1, markersize=5, linestyle='')
]
ax.legend(handles=legend_elements,bbox_to_anchor=(0, 0.1), loc='lower left',handlelength=0.5)

fig.tight_layout()

But we want to have more than two frequencies to be excited at the same time. Thus the problem is mapped on a graph. Each edge represents, that these frequencies can be excited at the same time without overlapping.

In [ ]:
graph = ms.get_graph(adjacency_mat)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(16*cm, 8*cm))
nx.draw_networkx(graph, pos=nx.circular_layout(sorted(graph.nodes(), reverse=True)), ax = ax, node_color='C0', font_color='w')
_ = ax.axis('off')
legend_elements = [
    plt.Line2D([0], [0], marker='', color='black',
               label='Possible\n frequency\n combination', alpha=1, markersize=5, linestyle='-',lw=1)
]
legend = ax.legend(handles=legend_elements,bbox_to_anchor=(0, 0), loc='lower left',handlelength=0.5)
legend.get_frame().set_alpha(0)
fig.tight_layout()

Finding groubs of frequencies that are connected is an NP-complete problem. Thus for high bandwidths long computation times and a lot of resouces are necessary.

In [ ]:
## Uncomment to analyse the memory and time consumption of the function get_cliques
# %load_ext memory_profiler
# %memit cliques = get_cliques(graph, f_s, f_lpf, harmonics_measurable, harmonics_expected)
# %time cliques = get_cliques(graph, f_s, f_lpf, harmonics_measurable, harmonics_expected)

cliques = ms.get_cliques(graph, f_s, f_lpf, harmonics_measurable, harmonics_expected,cliques_per_file=100)

In [ ]:
np.savetxt('cliques.csv', cliques, delimiter=',')

## Frequency selection

We want to have a frequency set that covers a high bandwidth and is mostly log-like scaled. For different applications this should be changed

In [ ]:
ssr_list = []
bandwidth_list = []

for frequencies in cliques:
    frequencies = np.array(frequencies, dtype=float)
    frequencies = np.sort(frequencies)
    
    # linear regression
    f_logs = np.log10(frequencies)
    indices = np.arange(1, len(frequencies) + 1)
    coefficients, residuals, _, _, _ = np.polyfit(indices, f_logs, deg=1, full=True)
    slope, intercept = coefficients
    predicted_logs = intercept + slope * indices
    residuals = f_logs - predicted_logs
    SSR = np.sum(residuals**2)
    ssr_list.append(SSR)
    
    # covered bandwidth
    bandwidth_ratio = np.max(frequencies) - np.min(frequencies)
    bandwidth_list.append(bandwidth_ratio)

ssr_array = np.array(ssr_list)
bandwidth_array = np.array(bandwidth_list)

# Normalize SSR and Bandwidth Ratio
ssr_min = np.min(ssr_array)
ssr_max = np.max(ssr_array)
normalized_ssr = 1 - (ssr_array - ssr_min) / (ssr_max - ssr_min)  # Invert to make higher better

bandwidth_min = np.min(bandwidth_array)
bandwidth_max = np.max(bandwidth_array)
normalized_bandwidth = (bandwidth_array - bandwidth_min) / (bandwidth_max - bandwidth_min)

# Choose weights
w_log = 0.5
w_bandwidth = 0.5

composite_scores = w_log * normalized_ssr + w_bandwidth * normalized_bandwidth
best_index = np.argmax(composite_scores)
best_frequencies = np.sort(cliques[best_index])
best_score = composite_scores[best_index]

print(f"The best frequency set is: {best_frequencies} with a composite score of {best_score:.4f}")
print(f"The exspected frequencies are: {ms.get_harmonics(best_frequencies, harmonics_expected).reshape(1,-1).astype(int)[0].tolist()},")
print(f"and they will be maped on: {ms.get_alias_f(ms.low_pass_filter(ms.get_harmonics(best_frequencies, harmonics_expected), f_lpf), f_s).reshape(1,-1).astype(int)[0].tolist()}")

In [ ]:
ms.check_unique(ms.get_alias_f(ms.low_pass_filter(ms.get_harmonics(best_frequencies,harmonics_expected),f_lpf),f_s))

In [ ]:
np.savetxt("best_frequencies.csv", best_frequencies, delimiter=",",fmt='%s')